# Eigenvalues and eigenvectors
* QR iteration is widely considered to be one of the most important algorithms of the 20th century. Very efficient and reliable way to calculate eigenvalues numerically. Big focus of this chapter of the course reader.

### Aside: computing eigenvalues exactly is "impossible" 
* No direct methods like there are for solving $A x = b$ (exact if we assume no roundoff error). Only iterative methods. 
* However these methods are pretty good nowadays, converging to ROE within a few iterations. 
* The reason is that no exact solution to roots of a polynomial of degree 5 or higher
* This matters since eigenvalue problem is equivalent to solving a polynomial equation for the eigenvalues (characteristic equation) 

## Methods for computing a single eigenvalue 
### Power iteration
* $()^H$ denotes the conjugate transpose
* $A^k = X \Lambda^k X^{-1}=\sum_i \lambda_i^k x_i y_i^H$ where $y_i$ is the $i$-th column of $(X^{-1})^H$. 
* Assuming $\Vert \lambda_1 \Vert > \Vert \lambda_2 \Vert \geq ... \geq \Vert \lambda_n \Vert$ then $A^k \approx \lambda_1^k x_1 y_1^H$ for large enough $k$. 

The key insight is that if we multiply by some random vector $z$ then we reveal a way to compute the first eigenvector $x_1$: 
$$ A^k z\approx (\lambda_1^k y_1^H z) x_1$$

To normalize $x_1$ we simply take $\tilde{x}_1=(A^k z)/\Vert A^k z \Vert$. Then $\lambda_1$ is found from the definition of $\tilde{x}_1$ as an eigenvector.

In [1]:
using LinearAlgebra

A = [2 1; 1 2]
@show eigvals(A)
@show eigvecs(A);

eigvals(A) = [1.0, 3.0]
eigvecs(A) = [-0.7071067811865475 0.7071067811865475; 0.7071067811865475 0.7071067811865475]


In [7]:
q = randn(size(A,1))
q /= norm(q)

niter = 20
err = zeros(niter)

for iter = 1:niter
    z = A*q
    global ev = q'*z
    q = z/norm(z)
    err[iter] = abs(ev - maximum(eigvals(A)))
end

@show ev
@show z;

UndefVarError: UndefVarError: ev not defined

In [3]:
using PlotlyJS

plot(err,Layout(yaxis_type="log",xaxis_title="Iteration",yaxis_title="Error",width=400))

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5778869639279644104\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16613519395045415030\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5778869639279644104\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin, width, xaxis, and yaxis"

Can show through a simple proof that the rate of convergence is $O(\lambda_2/
\lambda_1)^k$.


### Inverse iteration
Last method is good but does not take into consideration any guesses for where the eigenvalue is nor does it help us find the other eigenvalues. Idea is to apply power iteration to the matrix $(A-\mu I)^{-1}$ whose largest eigenvalue is $1/(\lambda_i - \mu)$.

In [4]:
μ = 1.1

for iter = 1:niter
    z = (A-μ*I)\q
    q = z/norm(z)
    ev = q'*A*q
end
print(ev)

UndefVarError: UndefVarError: ev not defined

### Rayleigh quotient iteration

In [5]:
for iter = 1:niter
    z = (A-μ*I)\q
    q = z/norm(z)
    μ = q'*A*q # update our guess as we go to improve convergence
end
return μ

1.0000000000000002

## Basic QR iteration
* Will introduce in this section and develop it further in the succeeding section. 
* Can calculate all eigenvalues at the same time. 
* Here we assume that all the eigenvalues can be ordered (no equalities) 

### Orthogonal iteration
#### Main idea for $r = 2$
Pseudocode is as follows: 
* Start with two random orthogonal vectors $q_1$ and $q_2$
* Keep multiplying them by $A$
* Project $q_2$ onto space orthogonal to $q_1$
* Normalize both vectors

Clearly $q_1$ is undergoing power iteration and so is converging to the eigenvector $x_1$. 

Now, the update step for $q_2$ based on the above pseudocode is thus: 
$$ q_2^{k} \approx (I-x_1 x_1^T)A q_2^{(k-1)}$$
One way to view this update is that $q_2$ is undergoing power iteration for the matrix $(I-x_1 x_1^T)A$. We can show that this converges to the eigenvector $(I-x_1 x_1^T)x_2$ with associated eigenvalue $\lambda_2$. Note that $q_2$ isn't necessarily converging to $x_2$ but one can visually rationalize that it is converging to the same space that would be spanned by $x_1$ and $x_2$ (instead spanned by $q_1$ and $q_2$). Namely if $x_1$ and $x_2$ are orthogonal then the angle between them is 90 degrees but if their dot product is close to one we see from the above equation that $q_2$ approaches $x_2 - x_1$ which is approximately zero (i.e. a small angle between them). In other words the relative angle between $x_2$ and $q_2$ may vary but the algorithm always converges to the same subspace.  

We can use the fact that $\text{span}\{q_1, q_2\} \approx \text{span}\{x_1, x_2\}$ to find the eigenvalues. That approximation (holding at convergence) implies that there is some vector that can be rotated via $Q$ into each of the eigenvectors: $Q v_i \approx x_i$ for $i = 1,2$. Through a few lines can show that this gives rise to $Q^T A Q v_i \approx \lambda_i v_i$ meaning $\lambda_1$ and $\lambda_2$ are eigenvalues of $Q^T A Q$. Since we proved that $Q^T A Q$ converges to an upper-triangular matrix (can show the bottom left entry must be zero) then these eigenvalues must be the diagonal entries of said matrix. 

#### Orthogonal iteration for general $r$ 
Will show that in orthogonal iteration $Q$ (a matrix we randomly initialize) converges to $Q^x$ where $(Q^x)^T A Q^x$ is upper-triangular (like in the case of $r=2$) and so eigenvalues on the diagonal. 

Note that currently we are considering arbitrary $r < n$ but in the case that $r = n$ then we actually obtain the Schur decomposition. 

We begin by considering the QR decomposition of the first $r$ eigenvectors:
$$ [x_1 | ... | x_r] = Q^x R^x$$
The *claim* is that $Q$ is converging to this $Q^x$. As before, it's clear that $q_1$ is converging to $x_1$ which is equal to $q_1^x$ based on the above QR decomposition. 

Now $\text{span}\{x_1, x_2\} = \text{span}\{q_1^x, q_2^x\}$. Why? We just showed $x_1$ is equal to $q_1^x$ to a sign flip and clearly from the QR decomposition $x_2$ is a linear combination of $q_1^x$ and $q_2^x$. Thus the space spanned by $q_1$ and $q_2$ converges to that spanned by $q_1^x$ and $q_2^x$. 

So $q_2$ is converging to something in the space spanned by $q_1^x$ and $q_2^x$ but it has to be by definition orthogonal to $q_1\approx q_1^x$. Thus it must be converging to the direction of $q_2^x$. This can be repeated for all columns via induction.  

#### Convergence to the Schur decomposition 
* Note that we use the notation $Q^x$ and $R^x$ to denote the QR decomposition for the eigenvector matrix $X$. 
* Things to keep in mind: 
    * Inverse of an upper-triangular matrix is itself upper-triangular 
    * Product of two UT matrices is itself UT. 
* For square $Q$ (i.e. $r=n$ we get $T = Q^T A Q$ (the schur decomposition). 

### QR iteration
* Yields the full Schur decomposition of a matrix: $T = Q^T A Q$ where $Q$ is orthogonal and $T$ is upper-triangular with eigenvalues on diagonal. 

### More methods to compute the eigendecomposition

## QR Iteration
We begin by noting that QR iteration works by updating $Q_k$ (the k-th iterate of $Q$ even though the eigenvalues show up on a different matrix $T_k = (Q_k)^T A Q_k$. Can we just calculate $T_{k+1}$ from $T_k$ somehow? 

Steps of QR iteration :
1. Compute QR decomposition of $T_k = U_{k+1} R_{k+1}$ where $T_k=Q_k^T A Q_k$. 
2. Update $T_{k+1} = R_{k+1} U_{k+1}$
Easiest way to run this algorithm is to initialize with $Q_0=I$. 

# Improvements to QR iteration 
As is, is not a great algorithm. QR factorization cost $O(n^3)$ flops and we have $O(n)$ iterations. 

## Starting with an upper Hessenberg matrix 
The brunt of the cost in the QR iteration comes from calculating the QR decomposition. Would be much faster if we had $A$ as upper Hessenberg, in which case we could simply apply Givens rotations (pg. 132 of CR). 
> In linear algebra, a Hessenberg matrix is a special kind of square matrix, one that is "almost" triangular. To be exact, an upper Hessenberg matrix has zero entries below the first subdiagonal, and a lower Hessenberg matrix has zero entries above the first superdiagonal. - *Wikipedia* 

Our goal is to:
1. Find $Q$ so that $H=Q^TAQ$
2. Run QR on H to find the Schur decomposition. 
Latter will take $O(n^3)$ so if we can do step one in $O(n^3)$ time then the total is $O(n^3)$. 

### Computing H

### Running QR iteration on H
- Comment notebook QR_Iteration_With_Shift.ipynb
- Finish section 3.2 


## QR with a shift strategy 